# Objective:

**We will be using the CoinGecko API to create one of the candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.**

## Installing APIs

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.3.0
    Uninstalling nbformat-5.3.0:
      Successfully uninstalled nbformat-5.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


## Importing Libraries


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

**Lets start off by getting the data we need. Using the <code>get_coin_market_chart_by_id(id, vs_currency, days)</code>. <code>id</code> is the name of the coin you want, <code>vs_currency</code> is the currency you want the price in, and <code>days</code> is how many days back from today you want.**


In [4]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

**The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.**


In [13]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1674943343906, 23017.957098506406],
 [1674946871248, 23002.99400172966],
 [1674950589617, 23008.07879887463],
 [1674954020298, 23125.088660679452],
 [1674957635489, 23253.831937559182]]

**Finally lets turn this data into a Pandas DataFrame.**


In [14]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
data

,TimeStamp,Price
0,1674943343906,23017.957099
1,1674946871248,23002.994002
2,1674950589617,23008.078799
3,1674954020298,23125.088661
4,1674957635489,23253.831938
...,...,...
719,1677520852402,23361.078020
720,1677524439404,23266.300082
721,1677528111409,23296.864190
722,1677531724095,23225.092570


**Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called <code>Date</code>. We will map our <code>unix_to_datetime</code> to each timestamp and convert it to a readable datetime.**


In [25]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))
data['date']


0      2023-01-29
1      2023-01-29
2      2023-01-29
3      2023-01-29
4      2023-01-29
          ...    
719    2023-02-28
720    2023-02-28
721    2023-02-28
722    2023-02-28
723    2023-02-28
Name: date, Length: 724, dtype: object

**Using this modified dataset we can now group by the <code>Date</code> and find the min, max, open, and close for the candlesticks.**


In [26]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

date         Price                                          
                         min           max         first          last
0   2023-01-29  23002.994002  23611.041600  23017.957099  23611.041600
1   2023-01-30  23051.808008  23907.560620  23605.333603  23167.907996
2   2023-01-31  22704.734249  23167.188227  23150.922102  23129.787553
3   2023-02-01  22883.572923  23189.694720  23139.129097  22988.904360
4   2023-02-02  23011.167834  24182.858615  23011.167834  23795.256836
5   2023-02-03  23379.056622  24047.164941  23892.327275  23557.806318
6   2023-02-04  23304.928359  23615.110490  23615.110490  23406.921314
7   2023-02-05  23091.639217  23451.116958  23444.870108  23104.751908
8   2023-02-06  22742.644242  23060.400193  22937.164358  22994.901677
9   2023-02-07  22786.483006  23102.669978  23058.911843  23010.330159
10  2023-02-08  22871.103325  23610.185929  23610.185929  22871.103325
11  2023-02-09  22526.668441  23017.385723  23017.385723  22553.628480
12  2023-02-10  21615.707322  22566.571700  22566.571700  21689.715252
13  2023-02-11  21499.661732  21801.324992  21704.843199  21713.467386
14  2023-02-12  21681.233182  22077.091347  21726.142061  22077.091347
15  2023-02-13  21504.989730  22067.786681  21994.156694  21504.989730
16  2023-02-14  21470.742475  22275.827533  21470.742475  22014.041737
17  2023-02-15  22070.836519  22772.573885  22070.836519  22772.573885
18  2023-02-16  22819.280899  24919.806558  22819.280899  24919.806558
19  2023-02-17  23666.448646  24960.697616  24898.768251  24067.364779
20  2023-02-18  24330.357197  24783.677812  24330.357197  24649.215097
21  2023-02-19  24584.204217  24941.421349  24646.503808  24811.629821
22  2023-02-20  24228.784427  24972.047000  24575.718477  24889.483885
23  2023-02-21  24425.082151  24982.455935  24868.743067  24425.082151
24  2023-02-22  23688.237676  24693.172169  24450.200388  23688.237676
25  2023-02-23  23736.359876  24631.747484  23736.359876  23849.993418
26  2023-02-24  23200.238166  24021.954249  23881.687373  23200.238166
27  2023-02-25  22967.064972  23252.486927  23126.905007  22998.513038
28  2023-02-26  22932.896357  23271.622484  23060.895078  23189.275207
29  2023-02-27  23220.629665  23757.151327  23220.629665  23438.387641
30  2023-02-28  23225.092570  23384.568306  23361.078020  23384.568306

**Finally we are now ready to use plotly to create our Candlestick Chart.**


In [27]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()